# Day 5: Supply II - Adding climate policies

We will now include climate policies more comprehensively (taxes and subsidies, but subsidies need to be paid by consumers).

The data and code are based on the paper "The Efficiency and Sectoral Distributional Implications of Large-Scale Renewable Policies," by Mar Reguant.

We first load relevant libraries, same as last session.

In [66]:
using Pkg
Pkg.add(["DataFrames", "CSV", "JuMP", "Ipopt", "Cbc", "HiGHS","Plots", "Printf"])

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [67]:
using DataFrames
using CSV
using JuMP
using Ipopt, Cbc, HiGHS
using Plots
using Printf

Remember to set your path correctly:

In [68]:
dirpath = "/Users/marreguant/Dropbox/TEACHING/BSE/Electricity2025/day5/practicum/"

"/Users/marreguant/Dropbox/TEACHING/BSE/Electricity2025/day5/practicum/"

## Building the model

We load the same data as last week, and also clean it up to simplify it further and create the demand and import curves.

In [69]:
dfclust = CSV.read(string(dirpath,"data_jaere_clustered.csv"), DataFrame);

# Re-scaling (we multiply by 8.76 to make it into a full year of hours (divided by 1000))
dfclust.weights = 8.76 * dfclust.weights / sum(dfclust.weights);

# Here only one demand type to make it easier
dfclust.demand = dfclust.q_residential + dfclust.q_commercial + dfclust.q_industrial;

# Calibrate demand based on elasticities (using 0.1 here as only one final demand)
elas = [.1, .2, .5, .3];
dfclust.b = elas[1] * dfclust.demand ./ dfclust.price;  # slope
dfclust.a = dfclust.demand + dfclust.b .* dfclust.price;  # intercept

# Calibrate imports (using elas 0.3)
dfclust.bm = elas[4] * dfclust.imports ./ dfclust.price;  # slope
dfclust.am = dfclust.imports - dfclust.bm .* dfclust.price;  # intercept

The technology file now includes the fixed cost of building new power plants (technologies 3-5). Note that we added an additional row for new natural gas plants.

We will use an annualization factor to pro-rate the importance of fixed costs for one year.

In [70]:
tech = CSV.read(string(dirpath,"data_technology.csv"), DataFrame);
afactor = (1 - (1 / (1.05^20.0))) / 0.05;
tech.F = tech.F ./afactor;

## Adding taxes and subsides to the problem

We modify our mixed integer code with an additional renewable subsidy and renewable charge that consumers need to pay.

In [ ]:
## Clear market based on first-order conditions
function clear_market_foc(data::DataFrame, tech::DataFrame; 
    ng_price = 3.5, tax=0.0, subsidy=0.0, renewable_charge=0.0)

    # We declare a model
    model = Model(
        optimizer_with_attributes(
            HiGHS.Optimizer, "output_flag" => false)
        );

    # Set useful indexes
    I = nrow(tech);  # number of techs
    T = nrow(data);  # number of periods
    M = 1e4;

    for i = 2:5
        tech.c[i] = tech.heatrate[i] * ng_price;
        tech.c2[i] = tech.heatrate2[i] * ng_price;
    end

    # Variables to solve for
    @variable(model, price[1:T]);
    @variable(model, demand[1:T]);
    @variable(model, imports[1:T]);
    @variable(model, quantity[1:T, 1:I] >= 0);
    @variable(model, shadow[1:T, 1:I] >= 0);  # price wedge if at capacity
    @variable(model, 0 <= K[5:I] <= 100.0);  # new capacity
    @variable(model, profit[5:I]); # tech annual profits, at most zero in equilibrium
    @variable(model, u1[1:T, 1:I], Bin);  # if tech used
    @variable(model, u2[1:T, 1:I], Bin);  # if tech at max
    @variable(model, u3[5:I], Bin);  # if tech is built

    @objective(model, Min, sum(price[t] * data.weights[t] / T for t=1:T));

    # Market clearing
    @constraint(model, [t=1:T], 
        demand[t] == data.a[t] - data.b[t] * (price[t] + renewable_charge));
    @constraint(model, [t=1:T], 
        imports[t] == data.am[t] + data.bm[t] * price[t]); #there are options for shifting imports like adding taxes.
    @constraint(model, [t=1:T], 
        demand[t] == sum(quantity[t,i] for i=1:I) + imports[t]);

    # Capacity constraints
    @constraint(model, [t=1:T], 
        quantity[t,1] <= u1[t,1] * data.hydronuc[t]); #we can only use the technology if u1 = 1
    @constraint(model, [t=1:T,i=2:4], 
        quantity[t,i] <= u1[t,i] * tech[i,"capUB"]);
    @constraint(model, [t=1:T, i=5:I], 
        quantity[t,i] <= u1[t,i] * 100.0);
    @constraint(model, [t=1:T], 
        quantity[t,5] <= K[5]);
    @constraint(model, [t=1:T], 
        quantity[t,6] <= K[6] * data.wind_cap[t]);
    @constraint(model, [t=1:T], 
        quantity[t,7] <= K[7] * data.solar_cap[t]);

    @constraint(model, [t=1:T], 
        quantity[t,1] >= u2[t,1] * data.hydronuc[t]); #if u2 = u1 = 1, hydronuc <= q <= hydronuc
    @constraint(model, [t=1:T,i=2:4], 
        quantity[t,i] >= u2[t,i] * tech[i,"capUB"]);
    @constraint(model, [t=1:T], 
        quantity[t,5] >= K[5] - M * (1.0-u2[t,5]));
    @constraint(model, [t=1:T], 
        quantity[t,6] >= K[6] * data.wind_cap[t] - M * (1.0-u2[t,6]));
    @constraint(model, [t=1:T], 
        quantity[t,7] >= K[7] * data.solar_cap[t] - M * (1.0-u2[t,7]));

    @constraint(model, [t=1:T,i=1:I], u1[t,i] >= u2[t,i]);

    # Constraints on optimality
    @constraint(model, [t=1:T,i=1:I],
        price[t] - tech.c[i] - tech.c2[i]*quantity[t,i] - tax * tech.e[i] + subsidy * tech.renewable[i] - shadow[t,i] 
        >= -M * (1.0-u1[t,i])); #the subsidy is increasing the shadow value renowable firms are getting. Leading to higher renewable entry
    @constraint(model, [t=1:T,i=1:I],
        price[t] - tech.c[i] - tech.c2[i]*quantity[t,i] - tax * tech.e[i] + subsidy * tech.renewable[i] - shadow[t,i] 
        <= 0.0);
    @constraint(model, [t=1:T,i=1:I], shadow[t,i] <= M*u2[t,i]);

    # Definition of profit
    @constraint(model, profit[5] == 
                        sum(data.weights[t]*shadow[t,5] for t=1:T) - tech.F[5]);
    @constraint(model, profit[6] == 
                        sum(data.weights[t]*shadow[t,6]*data.wind_cap[t] for t=1:T) - tech.F[6]);
    @constraint(model, profit[7] == 
                        sum(data.weights[t]*shadow[t,7]*data.solar_cap[t] for t=1:T) - tech.F[7]);

    # Constraints on investment 
    @constraint(model, [i=5:I], profit[i] <= 0.0); # zero profits if investing
    @constraint(model, [i=5:I], profit[i] >= -M*(1.0-u3[i])); # zero profits if investing
    @constraint(model, [i=5:I], K[i] <= M*u3[i]); # capacity only positive if firms can make zero profit

    # Solve model
    optimize!(model);

    status = @sprintf("%s", JuMP.termination_status(model));

    if (status=="OPTIMAL")
        p = JuMP.value.(price);
        avg_price = sum(p[t] * data.weights[t]/sum(data.weights) for t=1:T);		
        q = JuMP.value.(quantity);
        imp = JuMP.value.(imports);
        d = JuMP.value.(demand);
        cost = sum(data.weights[t] * (sum(tech.c[i] * q[t,i] + tech.c2[i] * q[t,i]^2 / 2 for i=1:I) + (imp[t] - data.am[t])^2/(2 * data.bm[t])) for t=1:T);
        subsidy_cost = sum(data.weights[t] * sum(subsidy * q[t,i] for i=6:7) for t=1:T);
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)),
            "avg_price" => avg_price,
            "cons_price" => avg_price + renewable_charge,
            "price" => p,
            "quantity" => q,
            "imports" => imp,
            "demand" => d,
            "shadow" => JuMP.value.(shadow),
            "profit" => JuMP.value.(profit),
            "u1" => JuMP.value.(u1),
            "u2" => JuMP.value.(u2),
            "u3" => JuMP.value.(u3),
            "gas_gw" => JuMP.value.(K[5]),
            "wind_gw" => JuMP.value.(K[6]),
            "solar_gw" => JuMP.value.(K[7]),            
            "emissions" => sum(data.weights[t] * sum(tech.e[i] * q[t,i] for i=1:I) for t=1:T),
            "subsidy_cost" => subsidy_cost,
            "needed_charge" => (subsidy_cost/sum(data.weights[t] * d[t] for t=1:T)),
            "wind_curtailment" => 1.0-sum(data.weights[t]*q[t,6]/(JuMP.value.(K[6])+0.0001) for t=1:T)/sum(data.weights[t]*data.wind_cap[t] for t=1:T),
            "solar_curtailment" => 1.0-sum(data.weights[t]*q[t,7]/(JuMP.value.(K[7])+0.0001) for t=1:T)/sum(data.weights[t]*data.solar_cap[t] for t=1:T));
        return results
    else
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)));
        return results
    end

end

clear_market_foc (generic function with 1 method)

In [72]:
results = clear_market_foc(dfclust, tech, ng_price=3.5, subsidy=15.0, renewable_charge=0.00)

Dict{String, Any} with 20 entries:
  "avg_price"         => 25.302
  "wind_curtailment"  => 2.90429e-6
  "price"             => [36.3469, 29.4818, 32.286, 37.2189, 15.8767, 35.8254, …
  "gas_gw"            => -0.0
  "profit"            => 1-dimensional DenseAxisArray{Float64,1,...} with index…
  "solar_curtailment" => 1.0
  "status"            => "OPTIMAL"
  "u1"                => [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1…
  "quantity"          => [9.46003 7.5 … 10.9352 -0.0; 4.0713 7.5 … 8.83915 0.0;…
  "solar_gw"          => 0.0
  "imports"           => [8.46177, 7.73344, 8.41005, 6.84681, 5.10787, 7.75198,…
  "demand"            => [38.4206, 28.1439, 27.3091, 29.8925, 23.4302, 25.5824,…
  "shadow"            => [26.3469 10.556 … 51.3469 51.3469; 19.4818 3.6909 … 44…
  "u2"                => [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1…
  "u3"                => 1-dimensional DenseAxisArray{Float64,1,...} with index…
  "subsidy_cost"      => 1463.6
  "emissions"

In [73]:
results["subsidy_cost"]

1463.6000830418125

In [ ]:
results_nosubsidy = clear_market_foc(dfclust, tech, ng_price=3.5)

In [ ]:
# Compare the two prices
histogram(results["price"], alpha=0.2, label="Subsidy")
histogram!(results_nosubsidy["price"], alpha=0.2, label="No Subsidy")

## Computing the renewable charge

We would like to add a constraint that states that the subsidies given to firms (solar and wind) need to equal the payments made by consumers with the renewable charges:
```
    # Subsidy charge 
    @constraint(model, 
        sum(data.weights[t] * sum(subsidy * quantity[t,i] for i=6:7) for t=1:T) == renewable_charge * sum(data.weights[t] * demand[t] for t=1:T));
```

One computational issue is that this is what is called a non-linear equation (`demand` and `renewable_charge` multiply each other, making it harder to compute).

It is best to proceed with a search approach for the renewable charge. We will code it with a simple loop here (akin to the visual search we saw last week for entry).

We get intuition first without making it a function.

In [ ]:
let
    current_diff = 1.0;
    guess = 5.0;
    while (current_diff > 1e-2)
        res = clear_market_foc(dfclust, tech, renewable_charge=guess, subsidy=15.0);
        newguess = res["needed_charge"];
        current_diff = (guess-newguess).^2;
        guess = newguess;
    end
    guess
end

The result is telling us that the renewable charge should be about $8.643 per MWh consumed, we have found an equilibrium.

### Making it into a function

We create a function that will do the loop and return the optimal solution.

In [ ]:
function clear_market_equilibrium(data::DataFrame, tech::DataFrame; 
    ng_price=3.5, tax=0.0, subsidy=0.0, renewable_charge=0.0)

    current_diff = 1.0;
    guess = subsidy/3.0;
    while (current_diff > 1e-2)
        res = clear_market_foc(data, tech,
                ng_price=ng_price, tax=tax, subsidy=subsidy, renewable_charge=guess);
        if (res["status"]=="OPTIMAL")
            newguess = res["needed_charge"];
        else
            print(string("Model is ",res["status"]," at ",guess,"\n"))
            newguess = guess+1.0;
        end
        current_diff = (guess-newguess).^2;
        guess = newguess;
    end

    # we solve at the equilibrium to return results
    res = clear_market_foc(data, tech,
            ng_price=ng_price, tax=tax, subsidy=subsidy, renewable_charge=guess);

    return res

end

In [ ]:
res_eq = clear_market_equilibrium(dfclust,tech,subsidy=15.0,ng_price=3.5,renewable_charge=10.0)

A subsidy of $15/MWh gets about 40 GW of installed new wind and average prices around $24/MWh.

What is an equivalent tax? How does it impact emissions and prices?

In [ ]:
results_tax = clear_market_foc(dfclust, tech, ng_price=3.5, tax=230.0)

## Follow-up questions

1. What are the costs of reaching a certain emissions target with subsidies vs. with taxes? You can solve this by using another loop that targets the right level of tax/subsidy.
   
2. What about adding three sectors (residential, commercial, industrial)? How do results change depending on which sector pays for the subsidies?
   
3. How could you add a Renewable Portfolio Standard or a Clean Energy Standard? Remember you need to search for the parameter that leads to a certain renewable share (RPS) or emissions rate (CES). This is the target in the loop.